In [38]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
from keras.models import load_model
import matplotlib.pyplot as pyplot
import pickle
from tqdm import notebook
import matplotlib.pyplot as plt
from keras.utils import to_categorical, plot_model


In [54]:
weights_path = '../weights/'
data_path='../data/'

## Define Discriminator

In [40]:
def define_discriminator(in_shape=(48,48,1), n_classes=7):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # scale up to image dimensions with linear activation
    n_nodes = in_shape[0] * in_shape[1]
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((in_shape[0], in_shape[1], 1))(li)
    # image input
    in_image = Input(shape=in_shape)
    # concat label as a channel
    merge = Concatenate()([in_image, li])
    
    be = Conv2D(32, (3,3), strides=(2,2), padding='same')(merge)
    be = LeakyReLU(alpha=0.2)(be)
    ce = Conv2D(64, (3,3), strides=(2,2), padding='same')(be)
    ce = LeakyReLU(alpha=0.2)(ce)
    fe = Conv2D(128, (3,3), padding='same')(ce)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Conv2D(128, (3,3), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Flatten()(fe)    
    fe = Dropout(0.2)(fe)
    out_layer = Dense(1, activation='sigmoid')(fe)
    
    model = Model([in_image, in_label], out_layer)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [41]:
plot_model(define_discriminator())

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

## Define Generator

In [5]:
def define_generator(latent_dim, n_classes=7):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    n_nodes = 12 * 12
    li = Dense(n_nodes)(li)
    li = Reshape((12, 12, 1))(li)
    
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    n_nodes = 128 * 12 * 12
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((12, 12, 128))(gen)
    
    # merge image gen and label input
    merge = Concatenate()([gen, li]) #(None, 12, 12, 129)
    
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
    gen = LeakyReLU(alpha=0.2)(gen)    
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    out_layer = Conv2D(1, (12,12), activation='tanh', padding='same')(gen)
    
    model = Model([in_lat, in_label], out_layer)
    return model

In [55]:
define_generator(100).summary()

NameError: name 'define_generator' is not defined

## Define GAN

In [7]:
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_label = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_label])
    
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_label], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

## Load real samples

In [3]:
# load face images
def load_real_samples():
   
    trainX=pickle.load(open(data_path+'trainX.pkl', 'rb'))
    trainy=pickle.load(open(data_path+'trainy.pkl', 'rb'))
    
    trainX = expand_dims(trainX, axis=-1)
    trainX = trainX.astype('float32')
    trainX = (trainX - 127.5) / 127.5
    return [trainX, trainy]

In [9]:
# # select real samples
def generate_real_samples(dataset, n_samples):
    images, labels = dataset
    index = randint(0, images.shape[0], n_samples)
    X, labels = images[index], labels[index]
    y = ones((n_samples, 1))
    return [X, labels], y

## Generate fake samples

In [10]:
def generate_latent_points(latent_dim, n_samples, n_classes=7):
    x_input = randn(latent_dim * n_samples)
    z_input = x_input.reshape(n_samples, latent_dim)
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

In [56]:
def generate_fake_samples(generator, latent_dim, n_samples):
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [images, labels_input], y

## Plot the results

In [ ]:
def plot(dloss1,dloss2,gloss,accuracy):
    clear_output(wait=True)
    plt.style.use("seaborn")

    epochs = range(len(dloss1))
    pltdloss1_x = [index for index,i in enumerate(dloss1) if i<2 ]
    pltdloss1_y = [i for i in dloss1 if i<2 ]
    plt.figure(figsize=(12,6))
    plt.plot(pltdloss1_x, pltdloss1_y, 'bo', label='Loss on real images')

    pltdloss2_x = [index for index,i in enumerate(dloss2) if i<2 ]
    pltdloss2_y = [i for i in dloss2 if i<2 ]
    plt.plot(pltdloss2_x, pltdloss2_y, 'ro', label='Loss on fake images')
    plt.title('Loss plots for Discriminator and Generator')
    plt.legend()
    
    plt.figure(figsize=(12,6))
    plt.plot(epochs, gloss, 'r')
    plt.title('Loss plot for CGAN, with min(loss):'+ str(round(min(gloss),2))+' for epoch:'+str(np.argmax(gloss)))
    plt.legend()

    plt.figure(figsize=(12,6))
    plt.plot(epochs, accuracy, 'g', label='Accuracy GAN')
    plt.title('Accuracy plot for CGAN, with max(acc):'+ str(round(max(accuracy),2))+' for epoch:'+str(np.argmax(accuracy)))
    plt.legend()
    plt.show()

## Train the GAN model

In [12]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=512):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    dloss1 = []
    dloss2 = []
    gloss = []
    accuracy = []
    # manually enumerate epochs
    for i in range(n_epochs):
        print('Epoch {}'.format(i))
        # enumerate batches over the training set
        tot_dloss1=0
        tot_dloss2=0
        tot_gloss=0
        tot_acc=0
        for j in notebook.tqdm(range(bat_per_epo)):
            # select a random half_batch of real images
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)      
            # generate a half_batch of "fake" images
            [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            
            # update discriminator model weights 
            d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real) 
            d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
            
            # prepare points in latent space as input for the generator
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            
            tot_dloss1+=d_loss1
            tot_dloss2+=d_loss2
            tot_gloss+=g_loss[0]
            tot_acc+=g_loss[1]

        dloss1.append(tot_dloss1/bat_per_epo)
        dloss2.append(tot_dloss2/bat_per_epo)
        gloss.append(tot_gloss/bat_per_epo)
        accuracy.append(tot_acc/bat_per_epo)
            
        # save the generator model every 5th epoch
        if i%5==0:
            g_model.save(weights_path+'cgan_epoch{}.h5'.format(i))
            
    g_model.save(weights_path+'cgan_epoch{}.h5'.format(i))

In [ ]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
train(g_model, 
      d_model, 
      gan_model, 
      dataset, 
      latent_dim,
      n_epochs=250)

Epoch 0


/usr/local/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



Epoch 1


### REFERENCES
1. https://machinelearningmastery.com/how-to-develop-a-conditional-generative-adversarial-network-from-scratch/